# SparkContext

In [1]:
from pyspark import SparkConf, SparkContext 
# https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.SparkContext.html

spark_conf = SparkConf().setAppName("spark-context-test")
spark = SparkContext(conf = spark_conf)
SparkConf().getAll()

24/05/23 12:15:27 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


[('spark.master', 'local[*]'),
 ('spark.submit.pyFiles', ''),
 ('spark.submit.deployMode', 'client'),
 ('spark.app.name', 'spark-context-test'),
 ('spark.executor.extraJavaOptions',
  '-Dio.netty.tryReflectionSetAccessible=true'),
 ('spark.ui.showConsoleProgress', 'true'),
 ('spark.driver.extraJavaOptions',
  '-Dio.netty.tryReflectionSetAccessible=true')]

# SparkSession

In [2]:
from pyspark.sql import SparkSession

spark = (
    SparkSession.builder
        .master("local")
        .appName("spark-session-test")
#         .config("spark.some.config.option", "some-value")
        .getOrCreate()
)
print(f"Hello, {spark.sparkContext.sparkUser()}")

# enableHiveSupport()

Hello, jovyan


# WordCount

In [4]:
spark = (
    SparkSession.builder
        .master("local")
        .appName("word-count")
        .getOrCreate()
)
sc = spark.sparkContext

logFile = "/home/jovyan/data/2024-05-19-14.json"
logData = sc.textFile(logFile).cache()

numAs = logData.filter(lambda s: 'a' in s).count()
numBs = logData.filter(lambda s: 'b' in s).count()

print(f"Lines with a: {numAs}, lines with b: {numBs}")

Lines with a: 196246, lines with b: 196246


# Read Data

In [5]:
df = spark.read.json("../data/2024-05-19-14.json")
df.printSchema()

root
 |-- actor: struct (nullable = true)
 |    |-- avatar_url: string (nullable = true)
 |    |-- display_login: string (nullable = true)
 |    |-- gravatar_id: string (nullable = true)
 |    |-- id: long (nullable = true)
 |    |-- login: string (nullable = true)
 |    |-- url: string (nullable = true)
 |-- created_at: string (nullable = true)
 |-- id: string (nullable = true)
 |-- org: struct (nullable = true)
 |    |-- avatar_url: string (nullable = true)
 |    |-- gravatar_id: string (nullable = true)
 |    |-- id: long (nullable = true)
 |    |-- login: string (nullable = true)
 |    |-- url: string (nullable = true)
 |-- payload: struct (nullable = true)
 |    |-- action: string (nullable = true)
 |    |-- before: string (nullable = true)
 |    |-- comment: struct (nullable = true)
 |    |    |-- _links: struct (nullable = true)
 |    |    |    |-- html: struct (nullable = true)
 |    |    |    |    |-- href: string (nullable = true)
 |    |    |    |-- pull_request: struct (nul

24/05/23 12:20:51 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


In [6]:
df.count()

196246

In [8]:
df.show(1, False)

+--------------------------------------------------------------------------------------------------------------------------+--------------------+-----------+----+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+------+------------------------------------------------------------------------------------------------+---------+
|actor                                                                                                                     |created_at          |id         |org |payload                                                                                                       

In [15]:
df = spark.read.format('org.apache.spark.sql.json') \
        .load("../data/2024-05-19-14.json")
df.show(3)

+--------------------+--------------------+-----------+----+--------------------+------+--------------------+---------+
|               actor|          created_at|         id| org|             payload|public|                repo|     type|
+--------------------+--------------------+-----------+----+--------------------+------+--------------------+---------+
|{https://avatars....|2024-05-19T14:00:00Z|38509489462|null|{null, d682fc58a4...|  true|{489717552, Drunk...|PushEvent|
|{https://avatars....|2024-05-19T14:00:00Z|38509489467|null|{null, ed0fe19aa1...|  true|{791822148, allen...|PushEvent|
|{https://avatars....|2024-05-19T14:00:00Z|38509489469|null|{null, e36c65c1a3...|  true|{802788764, carmi...|PushEvent|
+--------------------+--------------------+-----------+----+--------------------+------+--------------------+---------+
only showing top 3 rows



In [ ]:
# df2 = spark.read.json(
#     ['../data/2024-05-19-14.json','../data/2024-05-19-13.json'])
# df2.show(3)  

In [10]:
df.dtypes

[('actor',
  'struct<avatar_url:string,display_login:string,gravatar_id:string,id:bigint,login:string,url:string>'),
 ('created_at', 'string'),
 ('id', 'string'),
 ('org',
  'struct<avatar_url:string,gravatar_id:string,id:bigint,login:string,url:string>'),
 ('payload',
  'struct<action:string,before:string,comment:struct<_links:struct<html:struct<href:string>,pull_request:struct<href:string>,self:struct<href:string>>,author_association:string,body:string,commit_id:string,created_at:string,diff_hunk:string,html_url:string,id:bigint,in_reply_to_id:bigint,issue_url:string,line:bigint,node_id:string,original_commit_id:string,original_line:bigint,original_position:bigint,original_start_line:bigint,path:string,performed_via_github_app:struct<created_at:string,description:string,events:array<string>,external_url:string,html_url:string,id:bigint,name:string,node_id:string,owner:struct<avatar_url:string,events_url:string,followers_url:string,following_url:string,gists_url:string,gravatar_id:str

In [9]:
from pyspark.sql.types import StructType, StructField, StringType, DateType, LongType, BooleanType
# https://spark.apache.org/docs/latest/sql-ref-datatypes.html

schema = StructType([
      StructField("actor", StringType(), True),
      StructField("created_at", DateType(), True),
      StructField("id", LongType(), False),
      StructField("org", StringType(), True),
      StructField("payload", StringType(), True),
      StructField("public", BooleanType(), True),
      StructField("repo", StringType(), True),
      StructField("type", StringType(), True)
  ])


In [10]:
df = spark.read.schema(schema).json("../data/2024-05-19-14.json")
df.printSchema()

root
 |-- actor: string (nullable = true)
 |-- created_at: date (nullable = true)
 |-- id: long (nullable = true)
 |-- org: string (nullable = true)
 |-- payload: string (nullable = true)
 |-- public: boolean (nullable = true)
 |-- repo: string (nullable = true)
 |-- type: string (nullable = true)



In [11]:
rdd = sc.textFile('../data/2024-05-19-14.json')
rdd.take(1)

['{"id":"38509489462","type":"PushEvent","actor":{"id":83618436,"login":"Drunkula","display_login":"Drunkula","gravatar_id":"","url":"https://api.github.com/users/Drunkula","avatar_url":"https://avatars.githubusercontent.com/u/83618436?"},"repo":{"id":489717552,"name":"Drunkula/twitchtoolsglitch","url":"https://api.github.com/repos/Drunkula/twitchtoolsglitch"},"payload":{"repository_id":489717552,"push_id":18490096435,"size":1,"distinct_size":1,"ref":"refs/heads/master","head":"ae1b25f136b195ca0e671853c82a2000fd603442","before":"d682fc58a487a4e7fb0ca4f1b75145704447819d","commits":[{"sha":"ae1b25f136b195ca0e671853c82a2000fd603442","author":{"email":"spudguzzler@sharklasers.com","name":"Drunkula"},"message":"command replacements for players","distinct":true,"url":"https://api.github.com/repos/Drunkula/twitchtoolsglitch/commits/ae1b25f136b195ca0e671853c82a2000fd603442"}]},"public":true,"created_at":"2024-05-19T14:00:00Z"}']

# Write Data

In [12]:
df.select('type').distinct().collect()

[Row(type='PullRequestReviewEvent'),
 Row(type='PushEvent'),
 Row(type='GollumEvent'),
 Row(type='ReleaseEvent'),
 Row(type='CommitCommentEvent'),
 Row(type='CreateEvent'),
 Row(type='PullRequestReviewCommentEvent'),
 Row(type='IssueCommentEvent'),
 Row(type='DeleteEvent'),
 Row(type='IssuesEvent'),
 Row(type='ForkEvent'),
 Row(type='PublicEvent'),
 Row(type='MemberEvent'),
 Row(type='WatchEvent'),
 Row(type='PullRequestEvent')]

In [13]:
df2 = df.filter(df.type == 'CommitCommentEvent')
df2.count()

228

In [14]:
df2.write \
  .option("compression", "gzip") \
  .mode('Overwrite') \
  .json("../data/output.json")

In [ ]:
# docker exec -it de-2024-spark-master-1 s park-submit --master spark://spark-master:7077 jobs/hello-world.py data/2024-05-19-14.json